In [1]:
#importando librerias en general
import pandas as pd
import numpy as np
from time import time

In [2]:
#cargando datasets
dataPCAtrain = pd.read_csv("PCA_train.csv") #datos en el espacio de las pca
dataNormaltrain = pd.read_csv("train.csv")
ytrain = np.ravel(pd.read_csv("train_labels.csv").values) #variable respuesta

In [52]:
ytrain

array([5, 5, 5, ..., 6, 5, 7])

In [7]:
tuple(np.concatenate((np.arange(100, 0, -25), [1])))

(100, 75, 50, 25, 1)

In [57]:
#importing sklearn modules
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV

In [24]:
# param_grid = {"n_estimators": ([2**i for i in range(6)] + [100, 200]),
#     "max_depth": (list(range(5, 11)) + [25, 50]),
#     "max_leaf_nodes": [4,5,6,7,8],
#     "max_features": ['auto', 'sqrt', 'log2'],
#     "criterion": ['gini', 'entropy']
# }

param_grid = {"n_estimators": ([2**i for i in range(6)] + [100, 200]),
    "max_depth": (list(range(5, 11)) + [25, 50]),
    "max_leaf_nodes": [4,5,6,7,8],
    "max_features": ['auto', 'sqrt', 'log2']
}

In [29]:
mod_PCA = RandomForestRegressor(random_state=42)
t1 = time()
gs_PCA = GridSearchCV(mod_PCA, param_grid, cv = 3, scoring = "neg_mean_squared_error", return_train_score= True)
gs_PCA.fit(dataPCAtrain, ytrain)
t1f = time() - t1

In [45]:
mean_time_PCA = gs_PCA.cv_results_['mean_fit_time']
mean_score_test_PCA = np.sqrt(-gs_PCA.cv_results_['mean_test_score'])
mean_score_train_PCA = np.sqrt(-gs_PCA.cv_results_['mean_train_score'])
best_score = np.sqrt(-gs_PCA.best_score_)

In [42]:
mod_Norm = RandomForestRegressor(random_state=42)
t2 = time()
gs_Norm = GridSearchCV(mod_Norm, param_grid, cv = 3, scoring = "neg_mean_squared_error", return_train_score= True)
gs_Norm.fit(dataNormaltrain, ytrain)
t2f = time() - t2

In [43]:
mean_time_Norm = gs_Norm.cv_results_['mean_fit_time']
mean_score_test_Norm = np.sqrt(-gs_Norm.cv_results_['mean_test_score'])
mean_score_train_Norm = np.sqrt(-gs_Norm.cv_results_['mean_train_score'])
best_score_Norm = np.sqrt(-gs_Norm.best_score_)

In [46]:
best_score, best_score_Norm

(0.6901795261320118, 0.6721699116481785)

In [48]:
list_PCA = {"mean_score_test": mean_score_test_PCA,
    "mean_score_train": mean_score_train_PCA}
list_Norm = {"mean_score_test": mean_score_test_Norm,
    "mean_score_train": mean_score_train_Norm}

In [49]:
df_PCA = pd.DataFrame(list_PCA)
df_Norm = pd.DataFrame(list_Norm)

In [50]:
df_PCA.to_csv("DatoscompPCA.csv")
df_Norm.to_csv("DatoscompNorm.csv")

In [51]:
mod_PCA = gs_PCA.best_estimator_
mod_Norm = gs_Norm.best_estimator_

In [52]:
ytrain = ytrain.reshape((1279, 1))

In [61]:
PCA_predict = mod_PCA.predict(dataPCAtrain)
mean_squared_error(ytrain, PCA_predict)
mean_absolute_error(ytrain, PCA_predict)
mean_absolute_percentage_error(ytrain, PCA_predict)

0.09527587332416398

In [62]:
Norm_predict = mod_Norm.predict(dataNormaltrain)
mean_squared_error(ytrain, Norm_predict)
mean_absolute_error(ytrain, Norm_predict)
mean_absolute_percentage_error(ytrain, Norm_predict)

0.09169760023980762

In [63]:
test_Norm = pd.read_csv("test_wine.csv")
test_labels = pd.read_csv("test_labels.csv").values.reshape((320,1))

In [72]:
predicttest_Norm = mod_Norm.predict(test_Norm)
d = {"Modelo_Completo": [np.sqrt(mean_squared_error(test_labels, predicttest_Norm)),
mean_absolute_error(test_labels, predicttest_Norm),
mean_absolute_percentage_error(test_labels, predicttest_Norm)]}

In [73]:
test_pca = pd.read_csv("test_pca.csv")
predicttest_PCA = mod_PCA.predict(test_pca)
d["Modelo_PCA"] =  [np.sqrt(mean_squared_error(test_labels, predicttest_PCA)),
mean_absolute_error(test_labels, predicttest_PCA),
mean_absolute_percentage_error(test_labels, predicttest_PCA)]

In [79]:
compfinaldf = pd.DataFrame(d, index = ["RMSE", "MAE", "MAPE"])
compfinaldf["Porcentaje_de_error"] = np.round(100*np.abs(compfinaldf.Modelo_Completo - compfinaldf.Modelo_PCA)/compfinaldf.Modelo_Completo, 2)
compfinaldf

,Modelo_Completo,Modelo_PCA,Porcentaje_de_error
RMSE,0.578407,0.611941,5.80
MAE,0.462858,0.492503,6.40
MAPE,0.086691,0.091648,5.72


In [80]:
compfinaldf.to_csv("Compfinal.csv")